In [3]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from time import process_time 
import pickle
import random
from pathlib import Path
import os
import sys
from sklearn import preprocessing
import gc
from scsskutils import *

Found config file! Loading it.
D:\Siddharth Data\PLAsTiCC\Raw Data
Data Location = D:\Siddharth Data\PLAsTiCC\Raw Data
Found pickle location!
Pickle Location = C:\Users\Sid\Desktop\Pickle


In [4]:
data_location

WindowsPath('D:/Siddharth Data/PLAsTiCC/Raw Data')

In [5]:
classes = np.array([6, 15, 16, 42, 52, 53, 62, 64, 65, 67, 88, 90, 92, 95, 99])
class_weight = {6: 1, 15: 2, 16: 1, 42: 1, 52: 1, 53: 1, 62: 1, 64: 2, 65: 1, 67: 1, 88: 1, 90: 1, 92: 1, 95: 1, 99: 1}
pb_wavelength = {0:360, 1:476, 2:621.5, 3:754.5, 4:870, 5:1015}
#https://www.lsst.org/sites/default/files/docs/sciencebook/SB_2.pdf - pg35, Table 2.1

In [6]:
def get_wtable(df):
    all_y = np.array(df['target'], dtype = 'int32')
    y_count = np.unique(all_y, return_counts=True)[1]
    wtable = np.ones(len(classes))
    for i in range(0, y_count.shape[0]):
        wtable[i] = y_count[i] / all_y.shape[0]
    return wtable

In [7]:
target_dict ={
    6: 0,
    15: 1,
    16: 2,
    42: 3,
    52: 4,
    53: 5,
    62: 6,
    64: 7,
    65: 8,
    67: 9,
    88: 10,
    90: 11,
    92: 12,
    95: 13,
    99: 14,
    991: 14,
    992: 14,
    993: 14,
    994: 14
}


# Training

In [8]:
df=pd.read_csv(fr"{data_location}\training_set.csv")

In [9]:
df2d=pd.read_csv(fr"{data_location}\training_set_metadata.csv")

In [10]:
wtable = get_wtable(df2d)

In [11]:
df2d["target_class"]=df2d.loc[:,["target"]].replace(target_dict)
df2d.drop(["ra","decl","gal_l","gal_b","target"],1,inplace=True)    #Should hostgal_specz be used or not? Hmmm

In [12]:
df =df.merge(df2d[["object_id","hostgal_photoz"]],on="object_id")

In [13]:
df["object_id"] = df["object_id"].astype(np.float64)

### AUGMENTATION FOR TRAIN

In [12]:
dfcopies=[]
df2dcopies=[]

In [13]:
replication_num=25

In [14]:
import numpy as np

In [15]:
for i in range(replication_num):
    dfcopy=df.copy()
    df2dcopy=df2d.copy()
    if i<=25:
        percentage_to_drop = np.random.uniform(10,30+(1.5*i/2))
    elif i<=45 and i>25:
        percentage_to_drop = np.random.uniform(10,30+(i/2))
    else:
        percentage_to_drop = np.random.uniform(30,55)
    drop_indices = np.random.choice(dfcopy.index, int(percentage_to_drop/100 * len(dfcopy.index)), replace=False)
    tot_objs=len(dfcopy.object_id.unique())
    dfcopy = dfcopy.drop(drop_indices).reset_index(drop=True)
    new_objs=len(dfcopy.object_id.unique())
    while new_objs!=tot_objs:
        dfcopy = dfcopy.drop(drop_indices).reset_index(drop=True)
        
    df2dcopy["aug_hostgal_photoz"] = np.random.normal(df2dcopy['hostgal_photoz'], df2dcopy['hostgal_photoz_err']/1.5)
    df2dcopy.loc[df2dcopy["aug_hostgal_photoz"]<0,["aug_hostgal_photoz"]] = 0
    df2dcopy.loc[df2dcopy["aug_hostgal_photoz"]>5,["aug_hostgal_photoz"]] = 5
    
    dfcopy = dfcopy.merge(df2dcopy[["object_id","aug_hostgal_photoz"]],on="object_id")
    dfcopy["dt"]=(1+dfcopy["hostgal_photoz"])/(1+dfcopy["aug_hostgal_photoz"])
    
    df2dcopy["hostgal_photoz"] = df2dcopy["aug_hostgal_photoz"]
    df2dcopy.drop("aug_hostgal_photoz",1,inplace=True)
    dfcopy["hostgal_photoz"] = dfcopy["aug_hostgal_photoz"]
    dfcopy.drop("aug_hostgal_photoz",1,inplace=True)
    
    dfcopy['flux'] = np.random.normal(dfcopy['flux'], dfcopy['flux_err']/1.5)
    dfcopy["mjd"]=dfcopy["mjd"]*dfcopy["dt"]
    dfcopy.drop("dt",1,inplace=True)
    
    dfcopy["object_id"]+=(i+1)/(replication_num+1)
    df2dcopy["object_id"]+=(i+1)/(replication_num+1)
    
    dfcopies.append(dfcopy)
    df2dcopies.append(df2dcopy)

In [16]:
dfcopies.append(df)
df2dcopies.append(df2d)

In [17]:
df = pd.concat(dfcopies,ignore_index=True)
df2d = pd.concat(df2dcopies,ignore_index=True)

In [18]:
del(dfcopies)
del(df2dcopies)

### End of Augmentation

In [19]:
df

,object_id,mjd,passband,flux,flux_err,detected,hostgal_photoz
0,6.150385e+02,59750.4229,2,-548.692532,3.622952,1,0.0
1,6.150385e+02,59750.4306,1,-824.696533,5.553370,1,0.0
2,6.150385e+02,59750.4383,3,-470.382758,3.801213,1,0.0
3,6.150385e+02,59750.4450,4,-387.056474,11.395031,1,0.0
4,6.150385e+02,59752.4070,2,-677.025032,4.041204,1,0.0
...,...,...,...,...,...,...,...
28171173,1.307798e+08,60555.9838,4,-39.881969,46.477093,0,0.0
28171174,1.307798e+08,60560.0459,1,14.894439,18.947685,0,0.0
28171175,1.307798e+08,60571.0225,5,30.593130,50.695290,0,0.0
28171176,1.307798e+08,60585.9974,4,-23.471439,44.819859,0,0.0


In [20]:
df2d

,object_id,ddf,hostgal_specz,hostgal_photoz,hostgal_photoz_err,distmod,mwebv,target_class
0,6.150385e+02,1,0.0000,0.000000,0.0000,NaN,0.017,12
1,7.130385e+02,1,1.8181,1.698186,0.2552,45.4063,0.007,10
2,7.300385e+02,1,0.2320,0.229102,0.0157,40.2561,0.021,3
3,7.450385e+02,1,0.3037,0.879650,1.1523,40.7951,0.007,11
4,1.124038e+03,1,0.1934,0.240077,0.0176,40.4166,0.024,11
...,...,...,...,...,...,...,...,...
204043,1.307400e+08,0,0.0000,0.000000,0.0000,NaN,0.013,8
204044,1.307558e+08,0,0.1725,2.560600,1.1146,46.6108,0.136,11
204045,1.307629e+08,0,0.0000,0.000000,0.0000,NaN,0.430,2
204046,1.307729e+08,0,0.0000,0.000000,0.0000,NaN,0.034,8


In [21]:
df = df.merge(df.loc[:,["object_id","mjd"]].groupby('object_id').first().rename(columns={"mjd":"mjd_start"}),on="object_id")
df["mjd"] = df["mjd"] - df["mjd_start"]
df.drop("mjd_start",1,inplace=True)

In [22]:
# df["mjd"]=df["mjd"]/100

In [23]:
df["mjd"] = df["mjd"]/(df["hostgal_photoz"]+1)

In [24]:
df["lsst_wavelength"]=df.loc[:,["passband"]].replace(pb_wavelength)

In [25]:
df["source_wavelength"]=df["lsst_wavelength"]/(df["hostgal_photoz"]+1)

In [26]:
df.drop("lsst_wavelength",1,inplace=True)

In [27]:
df["mjd_diff1"]=df.loc[:,["object_id","mjd"]].groupby('object_id').diff(periods=1)
df["mjd_diff2"]=df.loc[:,["object_id","mjd"]].groupby('object_id').diff(periods=-1)
df["mjd_diff1"].fillna(0,inplace=True)
df["mjd_diff2"].fillna(0,inplace=True)

In [28]:
flux_pow=np.log2(df.loc[:,["object_id","flux"]].groupby('object_id').max() - df.loc[:,["object_id","flux"]].groupby('object_id').min()).reset_index().rename(columns={"flux":"flux_pow"})

In [29]:
df = df.merge(flux_pow,on="object_id")

In [30]:
df["flux"]=df["flux"]/np.power(2,df["flux_pow"])
df["flux_err"]=df["flux_err"]/np.power(2,df["flux_pow"])

In [31]:
flux_pow["small_flux_pow"]=flux_pow["flux_pow"]/10

In [32]:
df2d = df2d.merge(flux_pow.loc[:,["object_id","small_flux_pow"]],on="object_id")

In [33]:
assert np.array_equal(df2d.object_id.unique(),df.object_id.unique())

In [34]:
assert len(df2d) == len(df.object_id.unique())

In [35]:
df2d.loc[df2d['hostgal_photoz'] > 0, 'nonzero_photoz'] = 1
df2d.loc[df2d['hostgal_photoz'] == 0, 'nonzero_photoz'] = 0

In [36]:
#band of zerrxy
df["bands"]=df["passband"]+1

In [37]:
df.drop([
#     "object_id",
    "mjd",
    "passband",
    "detected",           #Decide whether to drop or not
    "hostgal_photoz",
    "flux_pow"],
    1,
    inplace=True)

In [38]:
labels=df2d.target_class.to_numpy()
df2d.drop("target_class",1,inplace=True)

df2d.drop([
    "object_id",
#     "hostgal_specz",      #Decide whethere to drop or not
],
    1,
    inplace=True)

In [39]:
df

,object_id,flux,flux_err,source_wavelength,mjd_diff1,mjd_diff2,bands
0,6.150385e+02,-0.311036,0.002054,621.5,0.0000,-0.0077,3
1,6.150385e+02,-0.467494,0.003148,476.0,0.0077,-0.0077,2
2,6.150385e+02,-0.266645,0.002155,754.5,0.0077,-0.0067,4
3,6.150385e+02,-0.219410,0.006459,870.0,0.0067,-1.9620,5
4,6.150385e+02,-0.383783,0.002291,621.5,1.9620,-0.0077,3
...,...,...,...,...,...,...,...
28171173,1.307798e+08,-0.000724,0.000844,870.0,7.0049,-4.0621,5
28171174,1.307798e+08,0.000270,0.000344,476.0,4.0621,-10.9766,2
28171175,1.307798e+08,0.000556,0.000921,1015.0,10.9766,-14.9749,6
28171176,1.307798e+08,-0.000426,0.000814,870.0,14.9749,-2.0398,5


In [40]:
df2d.fillna(0,inplace=True)
df.fillna(0,inplace=True)

In [41]:
#Meta of zerrxy 
twod_data=df2d.to_numpy()

In [42]:
#Hist of zerrxy, except last column is bands
obj_change_index=np.where(df["object_id"][:-1].to_numpy() != df["object_id"][1:].to_numpy())[0]+1
tuplist=list(zip(np.insert(obj_change_index,0,0),obj_change_index))
dfarray=df.to_numpy()
list_of_data_arrays=[]
for tup in tuplist:
    list_of_data_arrays.append(dfarray[tup[0]:tup[1],1:])
list_of_data_arrays.append(dfarray[obj_change_index[-1]:,1:])
#Then sequence padsequences
#max_length=max(map(len,list_of_data_arrays))
#threed_data=sequence.pad_sequences(list_of_data_arrays, maxlen=max_length)

In [43]:
df.columns

Index(['object_id', 'flux', 'flux_err', 'source_wavelength', 'mjd_diff1',
       'mjd_diff2', 'bands'],
      dtype='object')

In [44]:
df2d.columns

Index(['ddf', 'hostgal_specz', 'hostgal_photoz', 'hostgal_photoz_err',
       'distmod', 'mwebv', 'small_flux_pow', 'nonzero_photoz'],
      dtype='object')

In [45]:
with open(fr"{pickle_location}\train_3d_pickle", "wb") as fp:   #Pickling
    pickle.dump(list_of_data_arrays, fp)

with open(fr"{pickle_location}\train_2d_pickle", "wb") as fp:   #Pickling
    pickle.dump(twod_data, fp)
    
with open(fr"{pickle_location}\train_label_pickle", "wb") as fp:   #Pickling
    pickle.dump(labels, fp)